<a href="https://colab.research.google.com/github/wisrovi/RedesNeuronales/blob/master/07MAIR-WilliamSteveRodriguezVillamizar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Nota sobre notebook (leer antes de ejecutar otra linea)

## 0.0. 
Tanto para el ejercicio de crear la red neuronal desde cero como para la red neuronal con transferencia de conocimiento se aumento el dataseet de datos con ImageGenerator en cuatro (4) veces el tamaño original [No se pudo aumentar más veces debido a que el entorno de ejecución por mi GPU no lo permitió], esto para que los modelos entrenados tuvieran más generalización en los datos y con esto tuviera mejor respuesta en los test y predict

## 0.1.
Antes de efectuar cualquier carga del programa se debe iniciar el sistema ejecutando todos los codigos del apartado 2. Iniciar Sistema

## 0.2. 
Para evaluar la red Neuronal creada desde cero se puede ir al punto 3.3 Cargar Modelo Red Neuronal Guardado (Nueva red neuronal desde cero)

## 0.3.
Para evaluar el modelo por transferencia de conocimiento puede ir directamente al apartado 4.5. Cargar Modelo Red Neuronal Guardado (Transferencia conocimiento)

## 0.4.
El Base Folder se puede modificar en el apartado 2.2. Conexion con Google y Drive

# 1. Bibliografia

In [0]:
#urlArchivos = "https://drive.google.com/drive/folders/1w2w5dAF269C646RBwwU33YLNUG4JRfh9?usp=sharing"

In [0]:
#!pip install tensorflow==2.0.0-alpha0

In [0]:
#https://github.com/jbagnato/machine-learning
#https://www.datacamp.com/community/tutorials/convolutional-neural-networks-python#cnn
#https://www.datacamp.com/community/blog/keras-cheat-sheet
#https://www.pyimagesearch.com/2018/12/24/how-to-use-keras-fit-and-fit_generator-a-hands-on-tutorial/
#https://www.pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation/
#https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/
#https://www.aprendemachinelearning.com/clasificacion-de-imagenes-en-python/

# 2. Iniciar Sistema

## 2.1. Importar Librerias

In [0]:
#instalo las librerias necesarias
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU

#importo las librerias que se van a necesitar
import pickle
import matplotlib.pyplot as plt
import numpy as np
from numpy import array
from numpy import expand_dims

from keras import models
from keras import layers

from sklearn.model_selection import train_test_split

from keras.utils import to_categorical #para que las salidas queden parametrizadas en binario

from sklearn.metrics import classification_report #para ver parametros 

from keras.applications.inception_v3 import decode_predictions #para ver varios items de prediccion

#para autoescalar la imagen al tamaño requerido por la red a reentrenar
from keras.applications.inception_v3 import preprocess_input

#Incrementar el número de ejemplos para evitar overfitting (junto con dropout)
#Generar datos a partir de los presentes, a través de transformaciones
from keras.preprocessing.image import ImageDataGenerator



# Para las librerias para la red neuronal
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

#callback para mejorar los entrenamientos
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback  

## 2.2. Conexion con Google y Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
###################################################################################################
BASE_FOLDER = '/content/gdrive/My Drive/Master IA/Redes Neuronales/'
###################################################################################################

In [0]:
#le indicamos al colaboratory que las importaciones de librerias propias se haran tambien desde el directorio base
import sys
sys.path.append(BASE_FOLDER)

## 2.3 Importar Librerias propias

In [0]:
from ModeloExterno import ModeloExterno

## 2.4. Funciones

In [0]:
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def getRam_y_Gpu(imprimir=False):
  process = psutil.Process(os.getpid())
  ram_available = psutil.virtual_memory().available
  ram_disponible = humanize.naturalsize( ram_available  )
  gpu_disponible = gpu.memoryFree
  if imprimir:
      print("Gen RAM Free: " + ram_disponible, " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
      print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu_disponible, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
  ram_available = ram_available / 1000000000
  ram_available = round(ram_available, 2)
  return (ram_available, gpu_disponible)

#Algunas funciones utiles
def plot_image(img, title=""):
    #plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.title(title)
    #plt.axis('off')
    plt.show()

def convertLabelsToNumbers(y_train):
    # Transformando las labels de texto a valores numéricos
    y = np.unique(y_train)
    mapping = { key : value for key,value in zip(y,range(len(y)))}
    processed_y = np.array([mapping[i] for i in y_train])
    #print(len(mapping))
    return processed_y

def DividirDatos_Train_Test_Predic(x, y, porcentaje=0.25, randomDiv=50):
    print("Datos para división recibidos")

    # dividir los datos entre training, test y predic
    X_train, X_test, y_train, y_test = train_test_split(
        x_train_all, processed_y, 
        test_size=porcentaje, 
        random_state=randomDiv)
    print("División todo el dataset en: train y test completa")

    X_test, X_predic, y_test, y_predic = train_test_split(
        X_test, y_test, 
        test_size=porcentaje, 
        random_state=randomDiv)
    print("División dataset test en: test y predict completa")
    
    #Hacemos el One-hot Encoding
    train_Y_one_hot = to_categorical(y_train)
    test_Y_one_hot = to_categorical(y_test)
    predic_Y_one_hot = to_categorical(y_predic)
    print("Conversión de salidas en categorias completa")

    return (X_train, train_Y_one_hot), (X_test, test_Y_one_hot), (X_predic, predic_Y_one_hot)

def plot_history(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['acc'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

def plot_pred(pred):
    plt.figure(figsize=(8, 2))
    classes = [c[1] for c in pred]
    probas = [c[2] for c in pred]
    y_pos = np.arange(len(classes))
    plt.barh(y_pos, probas, align='center')
    plt.yticks(y_pos, classes)
    plt.gca().invert_yaxis()
    plt.xlabel('Probability')
    plt.xlim(0, 1)

def DefineCallback(nameFileIfSave): #guardar el mejor modelo w.r.t en val_losses
    """
    Agradecimientos a Data Talks, 
    url: https://www.youtube.com/watch?v=Gl-N3xr5zLI
    """    
    #EarlyStopping, detener el entrenamiento una vez que su pérdida comienza a aumentar 
    early_stop = EarlyStopping(
        monitor='val_loss', 
        patience=5, #argumento de patience representa el número de épocas antes de detenerse una vez que su pérdida comienza a aumentar (deja de mejorar). 
        min_delta=0,  #es un umbral para cuantificar una pérdida en alguna época como mejora o no. Si la diferencia de pérdida es inferior a min_delta , se cuantifica como no mejora. Es mejor dejarlo como 0 ya que estamos interesados ​​en cuando la pérdida empeora.
        mode='auto')
        
    mcp_save = ModelCheckpoint(
        nameFileIfSave + 'model-{acc:03f}-{val_acc:03f}.h5', 
        monitor='val_loss',
        save_best_only=True, 
        mode='min')

    #ReduceLROnPlateau, que si el entrenamiento no mejora tras unos epochs específicos, reduce el valor de learning rate del modelo
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.1, 
        patience=5, 
        epsilon=1e-4, 
        mode='min')
    
    callbacks_list = [early_stop, reduce_lr, mcp_save]  
    return callbacks_list

#hago las conversiones a la imagen
dataAumentationApliqued = False
def ConverImageUsingImageDataGenerator(datagen, imagen, nombre, cantidadConversiones):
    listaConversiones = []
    listaEtiquetasCadaConversion = []
    imagen4dimensiones = expand_dims(imagenConvertir, 0)
    imagen3dimensiones = np.squeeze(imagen4dimensiones)
    listaConversiones.append(convertNumpyToList(imagen3dimensiones))

    iterator_datagen_train = datagen.flow(imagen4dimensiones, batch_size=1)

    for i in range(cantidadConversiones):
        batch = iterator_datagen_train.next()
        image = np.squeeze(batch[0])
        listaConversiones.append(convertNumpyToList(image))

    for i in range(len(listaConversiones)):
        listaEtiquetasCadaConversion.append(nombreImagen)
    

    return listaConversiones, listaEtiquetasCadaConversion

def convertNumpyToList(imagenNumpy):
    return list(imagenNumpy)

def convertListoToNumpy(imagenList):
    return array(imagenList)

def getModelTransferLearning(baseModel):
    model = models.Sequential()
    model.add(baseModel)

    model.add(Dropout(0.25)) #apagar un 25% de manera aleatoria para reducir la cantidad de parametros


    model.add(
        Conv2D(32,
              kernel_size=(3, 3),
              activation='relu',
              padding='same',
              input_shape=(128,128,3)
              )
        )

    model.add( Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'  ) )

    model.add(Dropout(0.25)) #apagar un 25% de manera aleatoria para reducir la cantidad de parametros

    #para aplanar todas las capas
    #esto se hace debido a que el modelo de VG16 nos da 4 dimensiones de salida, y con esto reducimos las dimensiones a dos: numero datos, numero clases
    model.add(Flatten()) 

    # conectarlo a nueva parte densa
    # Genero la capa final con la cantidad de neuronas segun la cantidad de opciones de clasificacion
    model.add(Dense(nClasses, activation='softmax')) 

    model.summary()

    # compilar el modelo
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [0]:
ramColab, GpuColab = getRam_y_Gpu(True)

## 2.5. Importar Dataseet y clases

In [0]:
#Cargo los archivos de clases y las imagenes
with open(BASE_FOLDER+"pickle_all_images_df-002.pickle", "rb") as input_file:
    x_train_all = pickle.load(input_file)
with open(BASE_FOLDER+"pickle_all_classes.pickle", "rb") as input_file:
    y_train_all = pickle.load(input_file)

number_of_classes = len(y_train_all)

print("Entradas: ",x_train_all.shape)
print("Salidas: ", number_of_classes)

In [0]:
#show a image
plot_image(x_train_all[15])

## 2.6. Repartir Datos entre train-test-predic

In [0]:
print("Cantidad imagenes: ", len(x_train_all))
print("Cantidad etiquetas: ", len(y_train_all))

In [0]:
processed_y = convertLabelsToNumbers(y_train_all)
salidasCategoricas = to_categorical(processed_y)

classesImageGenerator = np.unique(salidasCategoricas)
nClassesImageGenerator = len(salidasCategoricas[1])
print("Cantidad de categorias en las clases",nClassesImageGenerator)

In [0]:
imagen = 7
plot_image(x_train_all[imagen])
print(processed_y[imagen], y_train_all[imagen])

In [0]:
classes = np.unique(processed_y)
nClasses = len(classes)

(X_train, y_train), (X_test, y_test), (X_predic, y_predic) = DividirDatos_Train_Test_Predic(x_train_all, processed_y)

print("Training (66.98%)") 
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print() 
print("Test (22.1%)") 
print("X_test", X_test.shape)
print("y_test", y_test.shape)
print()
print("Predict (10.89%)") 
print("X_predic", X_predic.shape)
print("y_predic", y_predic.shape)

print()
if y_predic.shape[1] == y_test.shape[1] and y_test.shape[1] == y_train.shape[1]:
    print("Distribución correcta")
else:
    print("Fallo en la distribución")

In [0]:
x_all = x_train_all
y_all = train_Y_one_hot = to_categorical(processed_y)

## 2.7. Aumentar el set de datos con DataGenerator (Opcional pero mejora los modelos cuando se utiliza)

### 2.7.1 Creando el modelo generador de datos

In [0]:
#Creamos el datagen del ImageDataGenerator para definir que cambios podria tener la imagen original

datagen = ImageDataGenerator(
    rotation_range=25, # grados de rotacion aleatoria
    width_shift_range=0.15, # fraccion del total (1) para mover la imagen
    height_shift_range=0.15, # fraccion del total (1) para mover la imagen
    shear_range=0.05, # deslizamiento
    zoom_range=0.1, # rango de zoom
    horizontal_flip=True, # girar las imagenes horizontalmente (eje vertical)
    fill_mode='nearest', # como rellenar posibles nuevos pixeles
    channel_shift_range=0.2 # cambios aleatorios en los canales de la imagen
)

### 2.7.2.Aplicando el data aumentation en el dataset original

In [0]:
cantidadConversiones = 2
#probado y funcionando con valor de 4, pero se requiere una buena GPU
#porque es facil superar la RAM maxima, y cuando se supera el colab ya no funciona



#También se incluirá la imagen original (en la practica no se suele usar la imagen original, pero para tener un dataset más grande en esta oportunidad la incluiremos)


if dataAumentationApliqued == False:
    print("Cantidad imagenes iniciales: ", len(X_train))
    print("Cantidad etiquetas iniciales: ", len(y_train))
    print("Tipo datos iniciales: ", type(X_train))
    listadoImagenes = []
    listaEtiquetasImagen = []
    for imagenProcesar in range(len(X_train)):    
        imagenConvertir = X_train[imagenProcesar] #elijo la imagen a usar de todo el dataset
        nombreImagen = y_train[imagenProcesar]
        listaConversiones, listaEtiquetasCadaConversion = ConverImageUsingImageDataGenerator(datagen, imagenConvertir, nombreImagen, cantidadConversiones-1)
        #añado las conversiones a la lista con append
        for i in range(len(listaConversiones)):
            listadoImagenes.append(listaConversiones[i])
            listaEtiquetasImagen.append(listaEtiquetasCadaConversion[i])

    #convierto la lista nuevamente a numpy para entregarla a los modelos
    X_train = convertListoToNumpy(listadoImagenes)
    y_train = convertListoToNumpy(listaEtiquetasImagen)
    print()
    print("Cantidad imagenes finales: ", len(X_train))
    print("Cantidad etiquetas finales: ", len(y_train))
    print("Tipo datos finales: ", type(X_train))
    dataAumentationApliqued = True
else:
    print("No se puede aplicar dataAumentation varias veces, si desea usarlo nuevamente, vuelva a reiniciar el colab (disponible en el menú de 'Entorno de ejecución').")

In [0]:
ramColab, GpuColab = getRam_y_Gpu(False)
if ramColab < 4:
    print("La Ram disponible no es suficiente para los entrenamientos de la red neuronal.")
else:
    getRam_y_Gpu(True)

In [0]:
for imagen in range(10):
    titulo = str(processed_y[imagen]) + " - " + y_train_all[imagen]
    plot_image(x_train_all[imagen], titulo)

## 2.8. Repartir datos en valid y train

In [0]:
ramColab, GpuColab = getRam_y_Gpu(False)
if ramColab < 4:
    print("La Ram disponible no es suficiente para los entrenamientos de la red neuronal.")
else:
    getRam_y_Gpu(True)

In [0]:
print(X_train.shape, y_train.shape)

In [0]:
train_X, valid_X, train_label, valid_label = train_test_split(
        X_train, y_train, 
        test_size=0.2, 
        random_state=10)

print(X_train.shape, y_train.shape)
print(valid_X.shape, valid_label.shape)

In [0]:
ramColab, GpuColab = getRam_y_Gpu(False)
if ramColab < 4:
    print("La Ram disponible no es suficiente para los entrenamientos de la red neuronal.")
else:
    getRam_y_Gpu(True)

# 3. Nueva Red Neuronal

In [0]:
ramColab, GpuColab = getRam_y_Gpu(False)
if ramColab < 3:
    print("La Ram disponible no es suficiente para los entrenamientos de la red neuronal.")
else:
    getRam_y_Gpu(True)

In [0]:
nombreModeloCreado = "wisrovi-nuevoModelo"
modelExtern = ModeloExterno(nombreModelo=nombreModeloCreado, rutaBase=BASE_FOLDER)

## 3.1. Modelo Red Neuronal

In [0]:
print("********** Modelo *********")
#MaxPooling2D es una forma de reducir el número de parámetros en nuestro modelo deslizando un filtro de agrupación de 2x2 a través de la capa anterior 
#y tomando el máximo de los 4 valores en el filtro de 2x2.

new_model = Sequential()
new_model.add( Conv2D(32,  kernel_size=(3, 3), activation='relu',padding='same', input_shape=(128,128,3))  )
new_model.add( Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'  ) )
new_model.add(MaxPooling2D((2, 2),padding='same'))

new_model.add( Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'  ) )
new_model.add( Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'  ) )
new_model.add( Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'  ) )
new_model.add(MaxPooling2D((2, 2),padding='same'))

new_model.add( Conv2D(96, kernel_size=(3, 3), activation='relu', padding='same'  ) )
new_model.add( Conv2D(96, kernel_size=(3, 3), activation='relu', padding='same'  ) )
new_model.add(MaxPooling2D((2, 2),padding='same'))

new_model.add(Dropout(0.5)) #apagar un 50% de manera aleatoria para reducir la cantidad de parametros

new_model.add(Flatten()) #para aplanar todas las capas
new_model.add(Dense(96, activation='relu'))
new_model.add(Dropout(0.1))
new_model.add(Dense(96, activation='relu'))
new_model.add(Dense(128, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(nClasses, activation='softmax')) #Genero la capa final con la cantidad de neuronas segun la cantidad de opciones de clasificacion

new_model.summary()

new_model.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

modelExtern.GuardarModelo(new_model)

## 3.2. Entrenar el modelo

In [0]:
#Entreno el modelo
numeroEntrenamientos = 50 #Al final de cada entrenamiento guardo los mejores pesos
numeroEpocas = 11  #ciclos en los cuales evaluo los datos de entrenamiento para hallar los mejores pesos
oldModel = None
oldAccuaricy = 0
conteoFallosEncontrarUnBuenModelo = 0
for i in range(numeroEntrenamientos):     
    print()   
    print()
    print("Entrenamiento " + str(i + 1))

    history = new_model.fit(
        train_X, train_label, 
        batch_size=32, # cantidad de imágenes que se toman a la vez en memoria
        epochs=numeroEpocas, 
        verbose=1,
  
        validation_data=(valid_X, valid_label),
        callbacks=DefineCallback(nombreModeloCreado)
        )
        
    plot_history(history)
    
    test_eval = new_model.evaluate(X_test, y_test, verbose=1)
    print('Test loss:', test_eval[0])
    print('Test accuracy:', test_eval[1])

    if oldAccuaricy < test_eval[1]:
        #Buscamos que haya mejora en el modelo, si hay mejora, guardamos el modelo
        oldAccuaricy = test_eval[1]
        modelExtern.GuardarPesosModelo()
        oldModel = new_model
        conteoFallosEncontrarUnBuenModelo = 0
    else:
        #Buscamos que el modelo tenga mejora en por lo menos 3 intentos, si supera tres intentos de entrenamiento sin mejora, se cancela el entrenamiento
        conteoFallosEncontrarUnBuenModelo += 1
        print("No se ha encontrado mejora al modelo, por lo cual se conservan el modelo anterior.")
        new_model = oldModel
        if conteoFallosEncontrarUnBuenModelo > 3:
            break
    print("***************")    
    print()

In [0]:
#valido el modelo
test_eval = new_model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [0]:
modelExtern.GuardarPesosModelo()

## 3.3 Cargar Modelo Red Neuronal Guardado (Nueva red neuronal desde cero)

In [0]:
modeloCargado = modelExtern.CargarModeloGuardado(False)
modeloCargado.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [0]:
print("*********************************************************************")
print("*                                                                   *")
print("*                      Modelo Creado desde cero                     *")
print("*                                                                   *")
print("*********************************************************************")
print()
print()

#valido el modelo
test_eval = modeloCargado.evaluate(X_test, y_test, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy: %s %%' %( round(test_eval[1]*100, 2)))
print()

#valido el modelo 2
test_eval = modeloCargado.evaluate(X_predic, y_predic, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy: %s %%' %( round(test_eval[1]*100, 2)))
print()

#valido el modelo 3
test_eval = modeloCargado.evaluate(x_all, y_all, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy: %s %%' %( round(test_eval[1]*100, 2)))
print()

# 4. Reentrenar un modelo de ImageNet (usando VG16)

In [0]:
ramColab, GpuColab = getRam_y_Gpu(False)
if ramColab < 3:
    print("La Ram disponible no es suficiente para los entrenamientos de la red neuronal.")
else:
    getRam_y_Gpu(True)

In [0]:
nombreModeloTransferencia = "wisrovi-transferLearning"
modeloTrasnferenciaConocimiento = ModeloExterno(nombreModelo=nombreModeloTransferencia, rutaBase=BASE_FOLDER)

## 4.1 Cargar Modelo ImageNet VGG16

In [0]:
# VGG16, VGG19, ResNet50, 
#Xception, InceptionV3, InceptionResNetV2, 
#MobileNetV2, DenseNet, RasNet


#https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/

from keras.applications import VGG16 
conv_base = VGG16(weights='imagenet',
                 include_top=False, # Si se deben incluir o no las capas de salida originales del modelo
                 input_shape=(128,128,3))

conv_base.summary()

## 4.2 Descongelar unas capas del Modelo ImageNet para reentrenarlas

In [0]:
congelarTodoModeloImagenet = False
if congelarTodoModeloImagenet:
  conv_base.trainable = False
else:
  # Freeze the layers except the last 5 layers
  cantidadCapasFinalesDescongelar = 5
  for layer in conv_base.layers[:-cantidadCapasFinalesDescongelar]:
      layer.trainable = False
      pass

  # Check the trainable status of the individual layers
  for layer in conv_base.layers:
      pass
      print(layer, layer.trainable)

conv_base.summary()

## 4.3. Crear nuestro modelo e incluirle el modelo de imageNet

In [0]:
pre_trained_model = getModelTransferLearning(conv_base)

modeloTrasnferenciaConocimiento.GuardarModelo(pre_trained_model)

## 4.4. Entrenar Modelo Con conocimiento Transferido

In [0]:
#Entreno el modelo

#Al final de cada entrenamiento guardo los mejores pesos
numeroEntrenamientos = 30

#ciclos en los cuales evaluo los datos de entrenamiento para hallar los mejores pesos
numeroEpocas = 15

#ver datos entrenamiento
print("Int: ", train_X.shape)
print("Out: ", train_label.shape)

oldModel = None
oldAccuaricy = 0
conteoFallosEncontrarUnBuenModelo = 0
for i in range(numeroEntrenamientos):     
    print()   
    print()
    print("Entrenamiento " + str(i + 1))

    history = pre_trained_model.fit(
        train_X, train_label, 
        batch_size=64, # cantidad de imágenes que se toman a la vez en memoria
        epochs=numeroEpocas,        
        verbose=1,
        validation_data=(valid_X, valid_label),
        callbacks=DefineCallback(nombreModeloTransferencia)
        )
        
    plot_history(history)   

    test_eval = pre_trained_model.evaluate(X_test, y_test, verbose=1)
    print('Test loss:', test_eval[0])
    print('Test accuracy:', test_eval[1])

    if oldAccuaricy < test_eval[1]:
        #Buscamos que haya mejora en el modelo, si hay mejora, guardamos el modelo
        oldAccuaricy = test_eval[1]
        modeloTrasnferenciaConocimiento.GuardarPesosModelo()
        oldModel = pre_trained_model
        conteoFallosEncontrarUnBuenModelo = 0
    else:
        #Buscamos que el modelo tenga mejora en por lo menos 3 intentos, si supera tres intentos de entrenamiento sin mejora, se cancela el entrenamiento
        conteoFallosEncontrarUnBuenModelo += 1
        pre_trained_model = oldModel
        print("No se ha encontrado mejora al modelo, por lo cual se conservan el modelo anterior.")
        if conteoFallosEncontrarUnBuenModelo > 3:
            break
    print("***************")    
    print()

## 4.5. Cargar Modelo Red Neuronal Guardado (Transferencia conocimiento)

In [0]:
modeloCargadoTransferenciaConocimiento = modeloTrasnferenciaConocimiento.CargarModeloGuardado(False)
modeloCargadoTransferenciaConocimiento.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [0]:
print("*********************************************************************")
print("*                                                                   *")
print("*           Modelo basado en transferencia de conocimiento          *")
print("*                                                                   *")
print("*********************************************************************")
print()
print()

#valido el modelo
test_eval = modeloCargadoTransferenciaConocimiento.evaluate(X_test, y_test, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy: %s %%' %( round(test_eval[1]*100, 2)))
print()

#valido el modelo 2
test_eval = modeloCargadoTransferenciaConocimiento.evaluate(X_predic, y_predic, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy: %s %%' %( round(test_eval[1]*100, 2)))
print()

#valido el modelo 3
test_eval = modeloCargadoTransferenciaConocimiento.evaluate(x_all, y_all, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy: %s %%' %( round(test_eval[1]*100, 2)))
print()

# 5. Reentrenar un modelo de ImageNet (usando xxxx)

In [0]:
ramColab, GpuColab = getRam_y_Gpu(False)
if ramColab < 3:
    print("La Ram disponible no es suficiente para los entrenamientos de la red neuronal.")
else:
    getRam_y_Gpu(True)

In [0]:
nombreModeloTransferencia2 = "wisrovi-transferLearning-2"
modeloTrasnferenciaConocimiento2 = ModeloExterno(nombreModelo=nombreModeloTransferencia2, rutaBase=BASE_FOLDER)

## 5.1. Cargar Modelo ImageNet MobileNetV2 

In [0]:
# VGG16, VGG19, ResNet50, 
#Xception, InceptionV3, InceptionResNetV2, 
#MobileNetV2, DenseNet, RasNet


#https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/

from keras.applications import MobileNetV2 
conv_base = MobileNetV2(weights='imagenet',
                 include_top=False, # Si se deben incluir o no las capas de salida originales del modelo
                 input_shape=(128,128,3))

conv_base.summary()

## 5.2. Descongelar unas capas del Modelo ImageNet para reentrenarlas

In [0]:
congelarTodoModeloImagenet = False
if congelarTodoModeloImagenet:
  conv_base.trainable = False
else:
  # Freeze the layers except the last 5 layers
  cantidadCapasFinalesDescongelar = 5
  for layer in conv_base.layers[:-cantidadCapasFinalesDescongelar]:
      layer.trainable = False
      pass

  # Check the trainable status of the individual layers
  for layer in conv_base.layers:
      pass
      print(layer, layer.trainable)

conv_base.summary()

## 5.3. Crear nuestro modelo e incluirle el modelo de imageNet

In [0]:
pre_trained_model_2 = getModelTransferLearning(conv_base)

modeloTrasnferenciaConocimiento2.GuardarModelo(pre_trained_model_2)

## 5.4. Entrenar Modelo Con conocimiento Transferido

In [0]:
#Entreno el modelo

#Al final de cada entrenamiento guardo los mejores pesos
numeroEntrenamientos = 30

#ciclos en los cuales evaluo los datos de entrenamiento para hallar los mejores pesos
numeroEpocas = 15

#ver datos entrenamiento
print("Int: ", train_X.shape)
print("Out: ", train_label.shape)

oldModel = None
oldAccuaricy = 0
conteoFallosEncontrarUnBuenModelo = 0
for i in range(numeroEntrenamientos):     
    print()   
    print()
    print("Entrenamiento " + str(i + 1))

    history = pre_trained_model_2.fit(
        train_X, train_label, 
        batch_size=32, # cantidad de imágenes que se toman a la vez en memoria
        epochs=numeroEpocas,        
        verbose=1,
        validation_data=(valid_X, valid_label),
        callbacks=DefineCallback(nombreModeloTransferencia2)
        )
        
    plot_history(history)   

    test_eval = pre_trained_model_2.evaluate(X_test, y_test, verbose=1)
    print('Test loss:', test_eval[0])
    print('Test accuracy:', test_eval[1])

    if oldAccuaricy < test_eval[1]:
        #Buscamos que haya mejora en el modelo, si hay mejora, guardamos el modelo
        oldAccuaricy = test_eval[1]
        modeloTrasnferenciaConocimiento2.GuardarPesosModelo()
        oldModel = pre_trained_model_2
        conteoFallosEncontrarUnBuenModelo = 0
    else:
        #Buscamos que el modelo tenga mejora en por lo menos 3 intentos, si supera tres intentos de entrenamiento sin mejora, se cancela el entrenamiento
        conteoFallosEncontrarUnBuenModelo += 1
        pre_trained_model_2 = oldModel
        print("No se ha encontrado mejora al modelo, por lo cual se conservan el modelo anterior.")
        if conteoFallosEncontrarUnBuenModelo > 3:
            break
    print("***************")    
    print()

## 5.5. Cargar Modelo Red Neuronal Guardado (Transferencia conocimiento)

In [0]:
modeloCargadoTransferenciaConocimiento2 = modeloTrasnferenciaConocimiento2.CargarModeloGuardado(False)
modeloCargadoTransferenciaConocimiento2.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [0]:
print("*********************************************************************")
print("*                                                                   *")
print("*          Modelo basado en transferencia de conocimiento 2         *")
print("*                                                                   *")
print("*********************************************************************")
print()
print()

#valido el modelo
test_eval = modeloCargadoTransferenciaConocimiento2.evaluate(X_test, y_test, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy: %s %%' %( round(test_eval[1]*100, 2)))
print()

#valido el modelo 2
test_eval = modeloCargadoTransferenciaConocimiento2.evaluate(X_predic, y_predic, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy: %s %%' %( round(test_eval[1]*100, 2)))
print()

#valido el modelo 3
test_eval = modeloCargadoTransferenciaConocimiento2.evaluate(x_all, y_all, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy: %s %%' %( round(test_eval[1]*100, 2)))
print()

# 6. Conclusiones

Al usar la transferencia de conocimiento es más facil alcanzar un modelo de clasificacion óptimo para el objetivo propuesto en pocas epocas de reentrenamiento

Al implementar un aumento de dataset con imageDataGenerator los modelos se entrenan con datos mas generales y esto logra que el modelo sea mas eficiente al predecir datos que no se mostraron durante el entrenamiento